# Applies Deep Learning to ePodium dataset for prediction of Dyslexia.

#### Import Packages

In [1]:
import mne
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import ipywidgets

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, BinaryAccuracy, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from functions import epodium, epodium_deep_learning, display_helper
from models.dnn import fully_connected_model
from models import transformer

import local_paths

#### Choose which processed data to use

In [2]:
processing_method_widget = ipywidgets.RadioButtons(options=['autoreject', 'ransac'], 
                                                   value='autoreject', 
                                                   description='processing:')
display(processing_method_widget)

RadioButtons(description='processing:', options=('autoreject', 'ransac'), value='autoreject')

<br>

## Preparing data iterator (Sequence) as input to the deep learning models.
https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

#### Split processed epochs* into train and test sequence.

*In the context of electroencephalography (EEG), *epochs* are EEG segments in which an event occurs. During processing, the epochs are chosen to be 1 second in which the event occurs at 0.2s. In the context of deep learning, *epochs* are iterations over the entire training dataset.

In [3]:
if(processing_method_widget.value == "autoreject"):
    path_processed = local_paths.ePod_processed_autoreject
if(processing_method_widget.value == "ransac"):
    path_processed = local_paths.ePod_processed_ransac

train, test = epodium_deep_learning.split_train_test_datasets(path_processed)
train_sequence = epodium_deep_learning.EvokedDataIterator(train, path_processed)
test_sequence = epodium_deep_learning.EvokedDataIterator(test, path_processed)

Analyzed: 228, bad: 42
186 files have enough epochs for analysis.
The dataset is split up into 138 train and 48 test experiments


<br>

## Deep Learning

The data is an *evoked* or *ERP* from a participant in the ePodium experiment. 60 EEG signals were averaged from -0.2 to +0.8 seconds after onset of an event. This is done for each of the 12 event types seperately.

__dimensions__: 
+ x (batches, timesteps, channels)
+ y (batches, labels)

__labels__: 
+ (Sex, At risk of dyslexia, first standard, standard, deviant)


#### Choose Deep Learning model

TODO check all trained models

In [4]:
model_widget = ipywidgets.RadioButtons(options=['fully_connected', 'transformer', 'new_transformer'],
                                       value='fully_connected', 
                                       description='Models:')
display(model_widget)

RadioButtons(description='Models:', options=('fully_connected', 'transformer', 'new_transformer'), value='full…

#### Train model

In [5]:
# Paths to save model info
base_path = os.path.join(local_paths.models, model_widget.value)
path_history = base_path + "_history.npy"
path_model = base_path + "_model"
path_testset = base_path + "_testset.txt"
path_weights = base_path + "_weights.h5"

# Train Model
print(f"Create model: {model_widget.value}")

# Save validation-set for future testing
with open(path_testset, 'w') as f:
    for participant in test:
        f.write(participant + '\n')

# Instantiate model
if(model_widget.value == "fully_connected"):
    model = fully_connected_model()
elif(model_widget.value == "transformer"):
    model = transformer.TransformerModel()
elif(model_widget.value == "new_transformer"):
    model = transformer.TransformerModel()

model.compile(optimizer=Adam(learning_rate=1e-3), loss=BinaryCrossentropy(), metrics=[Precision(), BinaryAccuracy(), Recall()])
checkpointer = ModelCheckpoint(filepath = path_weights, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=4, factor=0.5, verbose=1)

# Fit model
history = model.fit(x=train_sequence,
                    validation_data=test_sequence,
                    epochs=5,
                    callbacks=[checkpointer, reduce_lr])

np.save(path_history, history.history)
model.save(path_model)

Create model: fully_connected


2022-09-07 18:23:44.937558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 18:23:44.982979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 18:23:44.983262: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 18:23:44.985101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/5
18/18 [==============================] - ETA: 0s - loss: 0.6930 - precision: 0.5187 - binary_accuracy: 0.5127 - recall: 0.8359 
Epoch 1: val_loss improved from inf to 0.69349, saving model to /volume-ceph/floris_storage/models/fully_connected_weights.h5
18/18 [==============================] - 280s 16s/step - loss: 0.6930 - precision: 0.5187 - binary_accuracy: 0.5127 - recall: 0.8359 - val_loss: 0.6935 - val_precision: 0.4931 - val_binary_accuracy: 0.4931 - val_recall: 1.0000 - lr: 0.0010
Epoch 2/5
18/18 [==============================] - ETA: 0s - loss: 0.6923 - precision: 0.5221 - binary_accuracy: 0.5185 - recall: 0.8443
Epoch 2: val_loss did not improve from 0.69349
18/18 [==============================] - 199s 11s/step - loss: 0.6923 - precision: 0.5221 - binary_accuracy: 0.5185 - recall: 0.8443 - val_loss: 0.6937 - val_precision: 0.4931 - val_binary_accuracy: 0.4931 - val_recall: 1.0000 - lr: 0.0010
Epoch 3/5
18/18 [==============================] - ETA: 0s - loss: 0.692

#### Show Training History

In [9]:
display_helper.show_plot(x = range(len(history['loss'])), y = history['loss'] ,xlabel = "epochs", ylabel = "validation loss", title = "Loss during training")
display_helper.show_plot(x = range(len(history['loss'])), y = history['val_loss'] ,xlabel = "epochs", ylabel = "validation loss", title = "Validation loss during training")

TypeError: 'History' object is not subscriptable